In [ ]:
"""
This notebook gets the predictions on the methods in the test set from the fine-tuned model.
"""

In [1]:
!pip install transformers datasets evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform

In [5]:
import sys
def print_progress(percent):
    width = 50  # Width of the progress bar
    filled = int(width * percent / 100)
    bar = "#" * filled + "-" * (width - filled)
    sys.stdout.write(f"\r[{bar}] {percent}%")
    sys.stdout.flush()

In [2]:
# ------------------------------------------------------------------------
# Load Test Dataset and Create DatasetDict
# ------------------------------------------------------------------------
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict

test_df = pd.read_csv("ft_test_masked.csv")
test_df.drop(columns=["Unnamed: 0"], inplace=True)

test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "test": test_dataset
})

In [18]:
!unzip final_model.zip

Archive:  final_model.zip
replace final_model/added_tokens.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: final_model/added_tokens.json  
  inflating: final_model/config.json  
  inflating: final_model/generation_config.json  
  inflating: final_model/merges.txt  
  inflating: final_model/model.safetensors  
  inflating: final_model/special_tokens_map.json  
  inflating: final_model/tokenizer_config.json  
  inflating: final_model/training_args.bin  
  inflating: final_model/vocab.json  


In [19]:
# -----------------------------------
# Load fine-tuned model and tokenizer
# -----------------------------------

from transformers import T5ForConditionalGeneration, RobertaTokenizer

# Load model
model = T5ForConditionalGeneration.from_pretrained("final_model")

# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("final_model")

In [21]:
import pandas as pd

masked_functions = []
expected_if_statements = []
predicted_if_statements = []
predicted_full_methods = []
target_full_methods = []

for i in range(5000):
    # get model prediction
    input_text = dataset["test"][i]["masked_with_tab"]
    # input_text = dataset["test"][i]["masked_no_tab"]
    masked_functions.append(input_text)
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    output = model.generate(**inputs)
    predicted_if_statement = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted_if_statements.append(predicted_if_statement)

    # get expected if statement
    expected_if_statement = dataset["test"][i]["target_block"]
    expected_if_statements.append(expected_if_statement)

    try:
      # attach expected if statement back into the function
      assert("<MASK>" in input_text)
      predicted_method = input_text.replace("<MASK>", predicted_if_statement)
      predicted_full_methods.append(predicted_method)
      target_method = input_text.replace("<MASK>", expected_if_statement)
      target_full_methods.append(target_method)
      if i % 100 == 0:
        print_progress(((i+1)/5000)*100)
      # print(i)

    except AssertionError as e:
      print(i)
      raise e
print_progress(100)

[##################################################] 100%

In [22]:
# save predictions and other relevant data to df
df = pd.DataFrame({
    "input_function": masked_functions,
    "expected_if_statement": expected_if_statements,
    "predicted_if_statement": predicted_if_statements,
    "predicted_full_method": predicted_full_methods,
    "target_full_method": target_full_methods
})

# save df to csv
# df.to_csv("predictions_no_tab.csv")
df.to_csv("predictions_with_tab.csv")

In [23]:
df.head()

,input_function,expected_if_statement,predicted_if_statement,predicted_full_method,target_full_method
0,"def read ( self , count = True , timeout = Non...",if ignore_timeouts and is_timeout ( e ) :,if ignore_timeouts and is_noerr ( e ) :,"def read ( self , count = True , timeout = Non...","def read ( self , count = True , timeout = Non..."
1,"def _cache_mem ( curr_out , prev_mem , mem_len...",if prev_mem is None :,if mem_len < len ( curr_out ) :,"def _cache_mem ( curr_out , prev_mem , mem_len...","def _cache_mem ( curr_out , prev_mem , mem_len..."
2,def filtered ( gen ) : <TAB> for example in ge...,if example_len > max_length :,if example_len > max_length :,def filtered ( gen ) : <TAB> for example in ge...,def filtered ( gen ) : <TAB> for example in ge...
3,"def search ( self , query ) : <TAB> # ""Search....","if item . get ( ""type"" , """" ) == ""audio"" :","if ""guide_id"" in item :","def search ( self , query ) : <TAB> # ""Search....","def search ( self , query ) : <TAB> # ""Search...."
4,"def _check_script ( self , script , directive ...","if var . must_contain ( ""/"" ) :",if var . name in self . _untrusted_vars :,"def _check_script ( self , script , directive ...","def _check_script ( self , script , directive ..."
